In [ ]:
from mcts_node import MCTSNode as Node
from random import choice
from math import sqrt, log
from random_bot import think as expand
from rollout_bot import think as simulate

In [ ]:
import p3_t3
explore_factor = 2

b = p3_t3.Board()
state = b.starting_state()
b.display(state, None)
b.legal_actions(state)

In [ ]:
def expand_leaf(node, board, state):
    """ Adds a new leaf to the tree by creating a new child node for the given node.

    Args:
        node:   The node for which a child will be added.
        board:  The game setup.
        state:  The state of the game.

    Returns:    The added child node.

    """
    # action = expand(board, state) # use random_bot
    action = choice(node.untried_actions)
    node.untried_actions.remove(action)
    # temp_board = p3_t3.Board() # get action list?
    state = board.next_state(state, action)
    leaf = Node(node, action, board.legal_actions(state))
    node.child_nodes[action] = leaf

    return leaf
    # Hint: return new_node 

In [ ]:
def traverse_nodes(node, board, state, identity):  # Node, Board, State, player → Node
    """ Traverses the tree until the end criterion are met.

    Args:
        node:       A tree node from which the search is traversing.
        board:      The game setup.
        state:      The state of the game.
        identity:   The bot's identity, either 'red' or 'blue'.

    Returns:        A node from which the next stage of the search can proceed.

    """
    def UCT(node):
        return node.wins / (node.visits or 1) + explore_factor * sqrt(log(node.parent and node.parent.visits or 1)/(node.visits or 1))

    leaf = node
    while leaf.untried_actions and leaf.child_nodes:
        # alternate max/min based on identity?
        action = max(leaf.child_nodes.items(),
                     key=lambda l: UCT(l)).untried_actions
        leaf = leaf.child_nodes[action]
        state = board.next_state(state, action)
    return leaf
    # Hint: return leaf_node

In [ ]:
def rollout(board, state):  # Board, State → {1:(-1|1), 2:(-1|1)}
    """ Given the state of the game, the rollout plays out the remainder randomly.

    Args:
        board:  The game setup.
        state:  The state of the game.

    """
    while not board.is_ended():
        action = simulate(board, state)
        state = board.next_state(state, action)
    return board.points.values(state)

In [ ]:
def think(board, state):
""" Performs MCTS by sampling games and calling the appropriate functions to construct the game tree.

Args:
    board:  The game setup.
    state:  The state of the game.

Returns:    The action to be taken.

"""

    identity_of_bot = board.current_player(state)
    root_node = Node(parent=None, parent_action=None,
                    action_list=board.legal_actions(state))

    for step in range(num_nodes):
        print(step)
        print()
        # Copy the game for sampling a playthrough
        sampled_game = state

        # Start at root
        node = root_node

        # Do MCTS - This is all you! ------------------------------
        # Selection - Moving down the tree
        while node.child_nodes:
            node = traverse_nodes(            node, board, sampled_game, identity_of_bot)
            if node.parent_action:
                sampled_game.apply_move(node.parent_action)

        # Expansion - Add new child nodes to the last node we landed on & move there
        if node.untried_actions:
            node = expand_leaf(node, board, sampled_game)
            node = traverse_nodes(            node, board, sampled_game, identity_of_bot)

        # Simulation - Choose moves until result or predefined state is achieved aka rando time
        point = rollout(board, sampled_game)

        # print("win value:",board.win_values(sampled_game))
        # print("point:", point)
        # print("sampled game:",sampled_game)
        # print("IoB:",identity_of_bot)

        # Probably wrong
        # if board.win_values(sampled_game) == identity_of_bot:
        #     win = 1
        # else:
        #     win = 0
        win = board.win_values(sampled_game) == identity_of_bot

        # Update value back to the starting point
        backpropagate(node, win)

        # Compare actions ---------
        value = -1
        for i in root_node.child_nodes:
            # j has winrate and num. of visirs
            j = root_node.child_nodes[i]
            temp_value = (j.wins/j.visits)
        if value < temp_value and i:  # replace with better winrate
            value = temp_value
            final = i

    # Return an action, typically the most frequently used action (from the root) or the action with the best
    # estimated win rate.
    return final